# Smart Credit Bracket System
## Author: Chai Ke Rou
## Date: April 2023

### 1 - Introduction

The aim of this report is to investigate, visualise data and build machine learning models for the data that has been collected by a global finance company over the years. The datasets comprise basic bank details and a lot of credit-related information. In order to build an intelligent system to segregate the people into credit score brackets, exploratory analysis was performed to make predictions about the credit score based on the collected informations to minimize the manual efforts.

The report's rough outline is shown as follows:
1. Introduction
2. Data Auditing
3. Data Description
4. Descriptive Statistics
5. Supervised Learning
6. Classification
7. Kaggle Submission
8. Conclusion
9. References

### 2 - Data Auditing

In [ ]:
#Importing the necessary libraries and reading file(s)
%pip install numpy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
dataset = pd.read_csv('Credit-Scores-Dataset.csv')

In [ ]:
dataset.shape

In [ ]:
dataset.head(10)

In [ ]:
dataset.tail(10)

In [ ]:
missing_data = dataset.isnull().sum()
missing_data

There is no missing value in the dataset.

In [ ]:
dataset.dtypes

In [ ]:
df = dataset.groupby(['ID']).count()
df = df.sort_values('Age',ascending=False)
df.head()

In [ ]:
df = df.head(4)
merged_df = pd.merge(df, dataset, on='ID', how='left')
merged_df

Based on the aboved outputs, I noticed that there are multiple IDs in the dataset, however it seems like the multiple entries represent unique events or entities. Therefore, it is important to keep the duplicate entries in the dataset to avoid the loss of infomations. Nevertheless, we should take in consideration that ID doesn't really represents an unique entry in this case even though it is supposed to be. Hence, we shouldn't use ID to identify an unique entry.

### 3 - Data Description
The data was stored in a CSV file with the following variables:
1. ID: Unique identifier for an entry in the dataset
2. Month: Month in which data was recorded
3. Age: Age of the person
4. Occupation: Numerical representation of the individual's occupation
5. Annual_Income: Annual income of the person
6. Monthly_Inhand_Salary: Gross salary of the person
7. Num_Bank_Accounts: Number of bank account(s) the person holds
8. Num_Credit_Card: Number of credit cards the person possesses
9. Interest_Rate: Annual interest rate billed on the credit card's charges, also known as annual percentage rate (APR)
10. Num_of_Loan: Number of loans taken from the bank
11. Delay_from_due_date: Number of days from the last delayed payment to the payment due date
12. Num_of_Delayed_Payment: Number of delayed payment delayed by the person for Credit card and/or Loan
13. Changed_Credit_Limit: Change in credit card limit on a particular credit account (represented as a percentage or a number)
14. Num_Credit_Inquiries: Number of times your credit report has been inspected by lenders or creditors
15. Credit_Mix: Categorization of the different types of credit accounts (expressed as a series of numbers)
16. Outstanding_Debt: Total amount of outstanding debt up to the mentioned month
17. Credit_Utilization_Ratio: Percentage of a borrower’s total available credit that is currently being used
18. Credit_History_Age: Age of the person’s credit accounts
19. Payment_of_Min_Amount: Frequency of minimum payment(s) made over a period of time 
20. Total_EMI_per_month: Amount of money being paid each month towards loan or debt (Equated Monthly Installment(EMI))
21. Amount_invested_monthly: Amount of monthly investment by the person
22. Payment_Behaviour: Payment behaviour of the person 
23. Monthly_Balance: Monthly balance amount of the person
24. Credit_Score: Numerical representation of the individual's creditworthiness

There are in total 2100 data instances and 24 fields exist in the given dataset. Besides, all data collected are numbers, which include integer and float.

### 4 - Descriptive Statistics

In [ ]:
pd.set_option('display.max_columns', None) #to display all columns
dataset.describe()

In [ ]:
dataset.Credit_Score.value_counts(sort=True, ascending=False)

In [ ]:
dataset.Credit_Mix.value_counts(sort=True, ascending=False)

In [ ]:
dataset.Payment_Behaviour.value_counts(sort=True, ascending=False)

In [ ]:
dataset['Outstanding_Debt'].sum()

Summarize the key findings of the analysis as follows:
1. The maximum annual income of the customer is 179,987.28, however the minimum income is 7011.69. The average annual incomes among the people in the dataset is 51244.81.
2. The lowest gross pay is 319.56, whereas the highest monthly salary is 15101.94. The average monthly in-hand salary of the people in the dataset is 4255.55.
3. The average number of bank accounts and credit cards possessed by the people in the dataset is 5.47 and 5.57 respectively, it means that, on average, people in the dataset hold around 5 to 6 bank accounts and/or credit cards. 
4. The range of interest rates billed on the credit cards' charges is from 1% to 34%. The average rate is 14.53%.
5. The average number of loans taken by the people is 3.51. 
6. The average number of delayed payments is 13.40 and the average delay time for payments is 21.33 days.
7. The average of credit limit changes is 10.439481.
8. The maximum number of times the credit report has been inspected by lenders or creditors is 17.
9. Majority have standard types of credit accounts.
10. The total outstanding debt of the people in the dataset is 3,032,434.22.
11. The average credit utilization ratio for the people in the dataset is 21.36.
12. The minimum age of the people's credit accounts is 4 years, whilst maximum age is 403 years. 
13. People make at most one required payment on their credit card or loan account within a specific time frame. The average amount of monthly payments made by the people in the dataset is 138.53.
14. The average amount of monthly investments made by the people in the dataset is 56.30.
15. The most common payment behavior of the people is 201 which refers to low amount of spending and making small value payments.
16. The average monthly balance amount of the people is 399.34.
17. The distribution of credit scores among the people in the dataset is imbalanced, as most people get credit score 2, however there are only few credit score 3, and credit score 1 is the least popular one. 

### 5 - Supervised Learning

#### Supervised Machine Learning
Supervised machine learning is a type of machine learning that trains model on labelled datasets consisting of input features and corresponding output labels. The goal is to learn a model that can classify data or predict output labels accurately based on the input and labelled data for new unseen input data. Supervised learning can be split into two types of problems which are classification and regression. The difference between classification and regression is the output variable of classification is a category while output variable of regression is a real value.

#### Data Preprocessing (Replace outlier with sensible value)

In [ ]:
# Winsorization (clipping the extreme values beyond the 0.05th and 99.95th percentiles)
from scipy.stats import mstats

# Define Winsorization function
def winsorize_column(column):
    return mstats.winsorize(column, limits=[0.05, 0.05])

# Apply Winsorization to all columns using lambda function
winsorized_data = dataset.apply(lambda x: winsorize_column(x))

The reason that I choose to perform Winsorization using the scipy.stats module in Python to deal with the outliers in the dataset is because Winsorization function helps in decreasing the influence of outliers on the machine learning models by clipping the extreme values beyond specified percentiles. When outliers are present and have the potential to distort the distribution or impair the performance of the model, this function can be beneficial. Beside that, I have specified 0.05 as the limits as it is the most sensible and common used values that I found.

In [ ]:
from scipy.stats import skew

# calculate skewness coefficient of a dataset
dataset_skewness = skew(dataset)
winsorized_data_skewness = skew(winsorized_data)
print("Before data preprocessing: ",dataset_skewness,"\nAfter data preprocessing: ",winsorized_data_skewness)

Based on the skewness results, we can noticed that the skewness of the features incline to zero after the implementation of data preprocessing. This is also means that most of the outliers are replaced with approriate values through Winsorization, thus the impact of outliers to the skewness is reduced. After improving the data quality, we can then use this data to generate a machine learning model and facilitate meaningful insights. 

#### Labelled Data
Labelled data refers to a dataset for which the output label is known and provided to a machine learning algorithm as part of the training data. The input features and corresponding output labels are used to train an algorithm in order to make accurate predictions on new, unseen data. Data labelling is typically done by humans and can be a time consuming and expensive process. To sum it up, labelled data plays a vital role in supervised machine learning as it boosts the learning process of an algorithm.

In [ ]:
#the split of the features and labels
x = pd.DataFrame(winsorized_data.iloc[:, 0:23].values) # Features
y = winsorized_data.iloc[:, 23].values # Labelled Data: Credit Score

#### Training Set and Test Set
The training set is used to train the algorithm and tune its parameters, and the test set is used to evaluate the algorithm's performance using unseen data.
The training and test dataset should be part of the entire dataset and chosen randomly from the original dataset to avoid bias. Although the ratio of the training set to the test set varies in different scenarios, it is common to split the data 80% for training and 20% for testing. The training set is used to train the algorithms. After the algorithms done with the learning process, their performance is then evaluated on the test set. It is of paramount important to have the training and test dataset as to prevent overfitting. Otherwise, the algorithm might work well on the training dataset but not on the test dataset.

In [ ]:
#the split of the test and train dataset
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(
 x, y, test_size = 0.25, random_state = 0)

#### Feature Engineering

In [ ]:
#select the features through SelectKBest method
from sklearn.feature_selection import SelectKBest, f_classif

best_features= SelectKBest(score_func=f_classif, k=23)# k - The k best features that you want 
fit=best_features.fit(x,y)
df_scores= pd.DataFrame(fit.scores_)
df_columns= pd.DataFrame(dataset.columns)
features_scores= pd.concat([df_columns, df_scores], axis=1)
features_scores.columns= ['Features', 'Score']
top_feature=features_scores.sort_values(by = 'Score',ascending = False)[:23] #print up to kth element
top_feature 

In [ ]:
winsorized_data.corr()['Credit_Score'].sort_values()

I choose to implement feature engineering by using the scikit-learn SelectKBest class as it helps to minimise the dataset's dimensionality by keeping only the most useful features and therefore boost the performance of models.Those features are rated and analysed according to the scoring technique to decide which are the most informative. Moreover, 'f_classif' scoring method is chosen in this case as it is specific to classification tasks and evaluates the information gain of each feature. When each feature predicts the target variable, it calculates how much entropy is reduced or how much information is increased. Consequently, we can select those related features which are going to be used as input data to train the models. Based on the following output which indicates that the features with higher information gain values have higher relevance, I have chosen the top 14 features from Credit_Mix to Annual_Income as the input data because it provides the best performance for the SVC model. The reason that I choose to use SelectKBest instead of correlation method to find out the features with higher relevance is because only linear correlations between variables can be captured by corr(). SelectKBest, in comparison, offers greater flexibility and may capture a larger range of feature relevance by using a variety of scoring algorithms (e.g., ANOVA F-value, mutual information). 

### 6 - Classification

#### Difference between Binary and Multi-class Classification
Binary classification is a form of supervised machine learning to predict one of two possible classes for a given input. For instance, predicting whether an email is spam or not spam. The output of binary classification is a binary variable, such as 0 or 1, True or False, etc. In contrast, multi-class classification involves predicting one of three or more possible classes for a given input. To further illustrate this, it may be used in predicting the type of flower based on its petal length, width, etc. Multi-class classification can have three or more classes, and the output is a categorical variable. To sum it up, the main difference between binary and multi-class classification is the number of possible classes that can be predicted. Besides, the output, the methods used for evaluation and model selection may also differ between binary and multi-class classification.

#### Why we shall implement normalisation?
Normalisation which also known as feature scaling is crucial to be implemented in Support Vector Machine/Regression algorithms as they are sensitive to the scale of the input features. If the input features have different scales or ranges, the SVM algorithm may prioritize features with larger values and the performance may not be optimized. In our case, due to the range of values of the data varies widely, the range of all features should be normalized in order for each feature to contribute proportionately to the credit score output. By normalizing the data, the SVM algorithm can give equal importance to all the input features and thus, can make better predictions. Furthermore, it also helps to improve the speed of the training process and the overall performance of the model.

In [ ]:
#split the data again by using the features selected from feature engineering
X = winsorized_data.iloc[:, [14,8,13,10,17,15,18,7,11,6,9,12,22,4]].values #Input Data
Y = winsorized_data.iloc[:, 23].values # Labelled Data: Credit Score

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(
 X, Y, test_size = 0.25, random_state = 0)

In [ ]:
#normalisation
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

#### SVM (in relation to Linear Regression)
Support vector machines (SVMs) are a set of supervised learning methods used for classification, regression analysis and outlier detection. The dominant difference between SVM and linear regression is that SVM intends to find the hyperplane which separates the classes in the most appropriate way, while linear regression tries to fit a straight line that minimizes the sum of the squared errors between the predicted and actual values. Due to the kernel function in SVM which maps the input data into a higher-dimensional space, it is possible to separate the classes with a hyperplane[1]. The main aim of SVM is to find the best hyperplane which has the maximum distance from both the classes[1]. This can be done by finding different hyperplanes which classify the labels in the best way then the one which is farthest from the data points or the one which has a maximum margin will be chosen as the best hyperplane. Moreover, the SVM algorithm is also able to deal with non-linearly separable data through non-linear kernel functions. On the other hand, linear regression aims to fit a line that can be used to predict continuous output values based on the input features. This is considered as a regression algorithm that tries to reduce the sum of the squared errors between the predicted and actual values. Linear regression usually assumes that the relationship between the input and output variables is linear and tries to fit a line that best represents this relationship. Consequently, I choose to implement SVM(SVC) as we are supposed to classify data points into different credit score, and the classes are not linearly separable. Therefore, it is more suitable to use SVM rather than linear regression.

#### Kernel in SVM(SVC/ SVR)
A kernel is a function that enables the discovery of nonlinear decision boundaries between classes by transforming the input data into a higher dimensional space. Without having to explicitly determine the coordinates of the data in the higher dimensional space, the kernel function computes the dot product of two vectors in the transformed feature space. As a result, handling non-linear problems using the kernel method is computationally efficient. There are several kernel functions in SVM, such as linear, polynomial, radial basis function (RBF), sigmoid, anova and etc. It is crucial to choose a good kernel function as the performance of the model have a strong dependency on it. The choice of kernel often depends on the data and also the problem we are dealing with. To further elucidate this, the RBF kernel is usually used for non-linear problems, while the linear kernel is more appropriate for linearly separable data. The reason that I choose to use RBF kernel is that it is able to model complex, non-linear decision boundaries and therefore capture the non-linear relationships between the features and the target variable. As a result, it helps to enhance the performance of the SVC model and raise the accuracy of it.

In [ ]:
#SVC Model building
from sklearn.svm import SVC

svc = SVC(kernel='rbf') 
svc.fit(x_train, y_train)

#### Grid Search
In order to identify the optimum set of hyperparameters that produces the model's highest performance, I choose to implement Grid Search which is a method for determining a machine learning model's ideal hyperparameter values. A predetermined grid of hyperparameter combinations is methodically searched through and evaluated for each combination. Besides, I also specified the scoring method as 'qwk_scorer' as it is commonly used to evaluate the performance of models in tasks such as classification and also it is the scoring method being specified in this project.

In [ ]:
# Perform grid search to find the most suitable hyperparameter
from sklearn.metrics import make_scorer, cohen_kappa_score
from sklearn.model_selection import GridSearchCV
qwk_scorer = make_scorer(cohen_kappa_score)
param_grid = {'C': [1, 10, 100,2,20,200,3,30,300], 
              'gamma': [1, 0.1, 0.01,2,0.2,0.02,3,0.3,0.03]}

grid = GridSearchCV(estimator=svc, param_grid=param_grid, scoring=qwk_scorer, cv=5)
grid.fit(x_train, y_train)

# Print the best parameters and accuracy score
print("Best parameters: ", grid.best_params_)
print("Accuracy score: ", grid.best_score_)

In [ ]:
#Traing SVC model
from sklearn.svm import SVC
from sklearn.metrics import cohen_kappa_score

best_svc = SVC(kernel='rbf',C=2,gamma=0.03)#,C=2,gamma=0.03,C=20,gamma=0.01
best_svc.fit(x_train, y_train)
y_pred = best_svc.predict(x_test)

qwk_score = cohen_kappa_score(y_test, y_pred, weights='quadratic')
print("Quadratic Weighted Kappa score:", qwk_score)

#### Quadratic Weighted Kappa (QWK)
Quadratic Weighted Kappa, sometimes known as QWK, is a statistic that assesses how well two evaluations or ranks agree. It is frequently used in the assessment of machine learning models for multi-class classification issues especially when the classes are not always equally distributed. The degree of agreement between the predicted and actual classes, as well as the amount of variance that results from chance, are both taken into account by QWK. A QWK score is calculated as a result, ranging from -1 to 1, with 1 denoting perfect agreement and 0 denoting agreement no better than random. However, accuracy is a measure of how frequently the model makes accurate predictions by dividing the total number of prediction by the number of true predictions. Although it is a simple and straightforward metric, it comes with some drawbacks, especially when working with datasets that are unbalanced. For instance, a model that consistently predicts the majority class in an unbalanced dataset would achieve high accuracy, but it would not be a useful or instructive model. In conclusion, accuracy might not be the best option when the dataset is unbalanced or the cost of errors is not the same. QWK offers a more insightful assessment of model performance, especially when many evaluators are involved. 

#### Confusion Matrix
An confusion matrix, often referred to as a error matrix, is a performance assessment tool used in classification tasks. It provides an overview of a classification model's performance by displaying the quantity of true positives, true negatives, false positives, and false negatives. Based on the output, we can then construct a number of performance metrics, including accuracy, error, precision, specificity, sensitivity and negative predictive value which provide useful insight of the model's performance. When working with imbalanced classification, the matrix is extremely helpful. In our case, a matrix of size 3x3 is generated as it is a multiclass classification and there are total 3 different classes. 

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
d = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[1,2,3])
d.plot()

In [ ]:
from sklearn.metrics import classification_report
report = classification_report(y_test, y_pred)
print(report)

Credit score 2 has the highest precision 0.72, which indicates that when the model predicts credit score 2, it is correct around 72% of the time. Credit score 1 and credit score 3 have lower precision values of 0.73 and 0.51, respectively.

Furthermore, credit score 2 also has the highest recall (0.76), indicating that the model identifies around 76% of the actual instances of credit score 2 correctly. Credit score 3 has a relatively high recall of 0.71, while credit score 1 has the lowest recall of 0.49.

F1-score combines precision and recall into a single metric, considering both the ability to correctly identify positive instances and avoid false positives. Credit score 2 has the highest F1-score of 0.74, indicating a good balance between precision and recall. Credit score 1 and credit score 3 have lower F1-scores of 0.59 and 0.59, respectively.

Based on the output, we can come up with the result that the accuracy for the SVC model is 0.67, which represents the proportion of correctly classified instances out of all instances in the dataset. Although accuracy provides an overall measure of model performance, it may not reflect the performance for each individual class. The macro-average F1-score is 0.64, which is the average F1-score across all classes, giving equal weight to each class. The weighted average F1-score is also 0.67, which considers the class distribution in the dataset.

### 7 - Kaggle Submission

In [ ]:
from sklearn.preprocessing import StandardScaler
kaggle = pd.read_csv('Credit-Scores-Submission.csv')
x = pd.DataFrame(kaggle.iloc[:, [14,8,13,10,17,15,18,7,11,6,9,12,22,4]].values)
sc = StandardScaler()
x = sc.fit_transform(x)
y_pred = best_svc.predict(x) 

result = pd.read_csv('33363056-ChaiKeRou-v1.csv')
result['Credit_Score']= y_pred
# Write to CSV file
result.to_csv('33363056-ChaiKeRou-v1.csv', index=False)

result

### 8 - Conclusion


To put it in a nutshell, our analysis revealed several key findings. First, we identified a significant imbalance in the class distribution of our target variable, with only 29% of cases belonging to the credit score 1, while there are 54% belonging to the credit score 2 and 17% belonging to the credit score 3 according to the prediction made by the model. Second, we identified several important features for predicting credit score of a client, including credit mix, interest rate, num credit inquiries and etc. A person with more types of credit accounts, also low annual interest rate incurred on the credit
card’s charges and his/her credit report has rarely been accessed by lenders or creditors within a certain period of time will have a high credit score. Finally, our models achieved a high accuracy score of 0.67 and a QWK score of 0.50, indicating normal predictive power. 

I was given the chance to examine a real-world dataset for this data analysis project in order to gather insights and respond to research questions. Overall, I believe that I completed the job successfully and met the predetermined goals. This project taught me the value of data preprocessing, feature engineering, model selection, model training, model evaluation and etc. The dataset initially had a significant amount of outliers, which made it challenging to do insightful analysis. Nonetheless, making the best choice for the predictive model and hyperparameter to use presented another hurdle for me as numerous options are available. Throughout the project, I also try to explore several methods in order to enhance the performance of my model, such as OneClassSVM, Z-Score, Oversampling or Undersampling, computing and modifying class weight and etc. However, I found that the performance of the model declines after implementing those method. To further illustrate this, those methods might engender overfitting issue to the model and render in the decreasing of the performance, so I choose to remove them after finding this issue. Overall, I am glad with the project's results and the learnings I was able to draw from the data. I think this experience has helped me perform better at data analysis and will be helpful in the projects I work on in the future.

### 9 - References

1.https://www.analyticsvidhya.com/blog/2021/10/support-vector-machinessvm-a-complete-guide-for-beginners/